In [4]:
from libs import *

# Gemini API Usage

In [6]:
from dotenv import load_dotenv
load_dotenv()

if os.getenv('GOOGLE_API_KEY') is None:
    raise ValueError("GOOGLE_API_KEY environment variable not set")
else:
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

model = genai.GenerativeModel('gemini-1.5-flash')

def get_completion(
    prompt: str,
    max_tokens: Optional[int] = 2000,
    temperature: Optional[float] = 0.9,
    stop_sequences: Optional[List[str]] = None,
):
    """Get a text completion from Gemini API."""
    try:
        generation_config = genai.GenerationConfig(
            max_output_tokens=max_tokens,
            temperature=temperature,
            stop_sequences=stop_sequences,
        )
        response = model.generate_content(prompt, generation_config=generation_config)
        return response.text
    except Exception as e:
        logger.error(f"Error in get_completion: {e}")
        raise ValueError(f"Error in get_completion: {e}")


In [7]:

INFO_PROMPT = '''
You are an expert product analyzer. Your task is to analyze the image of a product and provide its description in JSON format with the following attributes:
1. **name**: The type of product, such as "rice" or "yoghurt".
2. **brand**: The brand of the product. If the brand is not visible or mentioned, set this value as `None`.
3. **type**: Categorize the product into one of these four types:
   - "Carbohydrates"
   - "Fruits and Vegetables"
   - "Protein"
   - "Fats"
4. **expiry_date**: The expiry date of the product in the format "YYYY-MM-DD". If the expiry date is not visible or mentioned, set this value as `None`.

Return ouput in json format:
{name: name, brand: brand, type: type, expiry_date: expiry_date}

'''


def get_information_products(img, prompt = INFO_PROMPT) -> Dict[str, Any]:
    """Get information products from Gemini API."""
    try:
        response = model.generate_content([prompt, img])
        text = response.text
        start_index = text.find('{')
        end_index = text.find('}') + 1
        json_str = text[start_index:end_index]
        try:
            product_info = json.loads(json_str)
            print(product_info)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
        
        return product_info
    except Exception as e:
        logger.error(f"Error in get_information_products: {e}")
        raise ValueError(f"Error in get_information_products: {e}")
    

# Food Management

In [8]:
from datetime import datetime

class Food_Management:
    def __init__(self, info: dict[str, Any]):
        self._name = info['name']
        self._brand = info['brand']
        self._type = info['type']
        self._expiry_date = info['expiry_date']
        self._used: bool = False
        self._out_of_stock: bool = False
        self._images = None
        self._info = None
        self._added_date = datetime.now().date()

    # Property for name
    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        self._name = value

    # Property for brand
    @property
    def brand(self):
        return self._brand

    @brand.setter
    def brand(self, value):
        self._brand = value

    # Property for type
    @property
    def type(self):
        return self._type

    @type.setter
    def type(self, value):
        self._type = value

    # Property for expiry_date
    @property
    def expiry_date(self):
        return self._expiry_date

    @expiry_date.setter
    def expiry_date(self, value):
        self._expiry_date = value

    # Method to get product information
    def get_info(self):
        return {
            'name': self._name,
            'brand': self._brand,
            'type': self._type,
            'expiry_date': self._expiry_date,
            'info': self._info
        }

    # Property to calculate days until expiry
    @property
    def days_until_expiry(self):
        if self._expiry_date:
            expiry = datetime.strptime(self._expiry_date, '%Y-%m-%d').date()
            today = datetime.today().date()
            delta = (expiry - today).days
            return delta
        return None

    # Prperty to determine expiry status
    @property
    def expiry_status(self):
        days_left = self.days_until_expiry
        if days_left is not None:
            if days_left < 0:
                return 'Red'  # expired
            elif days_left <= 3:
                return 'Yellow'  # expiring soon
            else:
                return 'Green'  # long shelf life
        return 'Unknown expiry status'
    
    @property
    def used(self):
        return self._used
    
    @used.setter
    def used(self, value):
        self._used = value

    @property
    def out_of_stock(self):
        return self._out_of_stock
    
    @out_of_stock.setter
    def out_of_stock(self, value):
        self._out_of_stock = value

In [9]:
img = PIL.Image.open('Yoghurt.jpg')
info = get_information_products(img)
A_1 = Food_Management(info)
print(A_1.get_info())
print(A_1.days_until_expiry)
print(A_1.expiry_status)
print(info)

{'name': 'Yoghurt', 'brand': 'Marigold', 'type': 'Protein', 'expiry_date': '2025-01-13'}
{'name': 'Yoghurt', 'brand': 'Marigold', 'type': 'Protein', 'expiry_date': '2025-01-13', 'info': None}
23
Green
{'name': 'Yoghurt', 'brand': 'Marigold', 'type': 'Protein', 'expiry_date': '2025-01-13'}


# Iventory Management

In [10]:
class Inventory_Management:
    def __init__(self):
        self.food_items: List[Food_Management] = []

    def add_food_item(self, food: Food_Management):
        """Add a Food_Management object to the inventory."""
        self.food_items.append(food)

    def sort_by_days_until_expiry(self):
        """Sort inventory by days left until expiry, ascending order."""
        self.food_items.sort(key=lambda food: (food.days_until_expiry or float('inf')))

    def sort_by_added_date(self):
        """Sort inventory by the date the food was added, ascending order."""
        self.food_items.sort(key=lambda food: food._added_date)

    def sort_by_category_and_expiry(self):
        """
        Groups food items by category and sorts each group by expiry date.
        
        Returns:
            Dict[str, List[Food_Management]]: Dictionary with categories as keys 
            and sorted food items as values.
        """
        categories = ["Carbohydrates", "Fruits and Vegetables", "Protein", "Fats"]
        sorted_items = {}
        
        for category in categories:
            sorted_items[category] = sorted(
                [item for item in self.food_items if item._type == category],
                key=lambda x: datetime.strptime(x._expiry_date, "%Y-%m-%d")
            )
        return sorted_items

    def display_category(self, category: str):
        """
        Displays all food items in the specified category in a readable format.
        """
        items_in_category = self.sort_by_category_and_expiry().get(category)
        if not items_in_category:
            print(f"No items found in the category: {category}")
        else:
            print(f"Food items in the category '{category}':")
            return [item.get_info() for item in items_in_category]

    def display_inventory(self):
        """Display the inventory in a readable format."""
        return [item.get_info() for item in self.food_items]


## Adding data

In [11]:
# Example usage
product1 = Food_Management({
    'name': 'Yoghurt',
    'brand': 'Marigold',
    'type': 'Protein',
    'expiry_date': '2025-01-13'
})

product2 = Food_Management({
    'name': 'Apple',
    'brand': None,
    'type': 'Fruits and Vegetables',
    'expiry_date': '2024-12-25'
})

product3 = Food_Management({
    'name': 'Rice',
    'brand': 'Golden Grain',
    'type': 'Carbohydrates',
    'expiry_date': '2026-05-01'
})

product4 = Food_Management({
    'name': 'Banana',
    'brand': None,
    'type': 'Fruits and Vegetables',
    'expiry_date': '2024-12-23'
})

product5 = Food_Management({
    'name': 'Chicken Breast',
    'brand': 'Farm Fresh',
    'type': 'Protein',
    'expiry_date': '2024-12-27'
})

product6 = Food_Management({
    'name': 'Olive Oil',
    'brand': 'Extra Virgin',
    'type': 'Fats',
    'expiry_date': '2025-07-15'
})

product7 = Food_Management({
    'name': 'Bread',
    'brand': 'Baker’s Delight',
    'type': 'Carbohydrates',
    'expiry_date': '2024-12-24'
})

product8 = Food_Management({
    'name': 'Milk',
    'brand': 'Dairy Pure',
    'type': 'Protein',
    'expiry_date': '2024-12-22'
})

product9 = Food_Management({
    'name': 'Carrot',
    'brand': None,
    'type': 'Fruits and Vegetables',
    'expiry_date': '2024-12-30'
})

product10 = Food_Management({
    'name': 'Eggs',
    'brand': 'Golden Eggs',
    'type': 'Protein',
    'expiry_date': '2024-12-29'
})

product11 = Food_Management({
    'name': 'Cheese',
    'brand': 'Cheddar Bliss',
    'type': 'Protein',
    'expiry_date': '2025-02-01'
})

product12 = Food_Management({
    'name': 'Potato',
    'brand': None,
    'type': 'Fruits and Vegetables',
    'expiry_date': '2025-01-10'
})

product13 = Food_Management({
    'name': 'Butter',
    'brand': 'Creamy Delight',
    'type': 'Fats',
    'expiry_date': '2025-04-20'
})


In [12]:
inventory = Inventory_Management()
inventory.add_food_item(product1)
inventory.add_food_item(product2)
inventory.add_food_item(product3)
inventory.add_food_item(product4)
inventory.add_food_item(product5)
inventory.add_food_item(product6)
inventory.add_food_item(product7)
inventory.add_food_item(product8)
inventory.add_food_item(product9)
inventory.add_food_item(product10)
inventory.add_food_item(product11)
inventory.add_food_item(product12)
inventory.add_food_item(product13)

## Testing Func

In [13]:
print("\nSorted by Days Until Expiry:")
inventory.sort_by_days_until_expiry()
for item in inventory.food_items:
    print(item.get_info())


Sorted by Days Until Expiry:
{'name': 'Milk', 'brand': 'Dairy Pure', 'type': 'Protein', 'expiry_date': '2024-12-22', 'info': None}
{'name': 'Banana', 'brand': None, 'type': 'Fruits and Vegetables', 'expiry_date': '2024-12-23', 'info': None}
{'name': 'Bread', 'brand': 'Baker’s Delight', 'type': 'Carbohydrates', 'expiry_date': '2024-12-24', 'info': None}
{'name': 'Apple', 'brand': None, 'type': 'Fruits and Vegetables', 'expiry_date': '2024-12-25', 'info': None}
{'name': 'Chicken Breast', 'brand': 'Farm Fresh', 'type': 'Protein', 'expiry_date': '2024-12-27', 'info': None}
{'name': 'Eggs', 'brand': 'Golden Eggs', 'type': 'Protein', 'expiry_date': '2024-12-29', 'info': None}
{'name': 'Carrot', 'brand': None, 'type': 'Fruits and Vegetables', 'expiry_date': '2024-12-30', 'info': None}
{'name': 'Potato', 'brand': None, 'type': 'Fruits and Vegetables', 'expiry_date': '2025-01-10', 'info': None}
{'name': 'Yoghurt', 'brand': 'Marigold', 'type': 'Protein', 'expiry_date': '2025-01-13', 'info': Non

In [14]:
Protein = inventory.sort_by_category_and_expiry().get("Protein")
Carbohydrates = inventory.sort_by_category_and_expiry().get("Carbohydrates")
Fats = inventory.sort_by_category_and_expiry().get("Fats")
Fruits_and_Vegetables = inventory.sort_by_category_and_expiry().get("Fruits and Vegetables")

In [15]:
for item in Protein:
    print(item.get_info())

{'name': 'Milk', 'brand': 'Dairy Pure', 'type': 'Protein', 'expiry_date': '2024-12-22', 'info': None}
{'name': 'Chicken Breast', 'brand': 'Farm Fresh', 'type': 'Protein', 'expiry_date': '2024-12-27', 'info': None}
{'name': 'Eggs', 'brand': 'Golden Eggs', 'type': 'Protein', 'expiry_date': '2024-12-29', 'info': None}
{'name': 'Yoghurt', 'brand': 'Marigold', 'type': 'Protein', 'expiry_date': '2025-01-13', 'info': None}
{'name': 'Cheese', 'brand': 'Cheddar Bliss', 'type': 'Protein', 'expiry_date': '2025-02-01', 'info': None}


In [16]:
for item in Carbohydrates:
    print(item.get_info())

{'name': 'Bread', 'brand': 'Baker’s Delight', 'type': 'Carbohydrates', 'expiry_date': '2024-12-24', 'info': None}
{'name': 'Rice', 'brand': 'Golden Grain', 'type': 'Carbohydrates', 'expiry_date': '2026-05-01', 'info': None}


In [17]:
for item in Fats:
    print(item.get_info())

{'name': 'Butter', 'brand': 'Creamy Delight', 'type': 'Fats', 'expiry_date': '2025-04-20', 'info': None}
{'name': 'Olive Oil', 'brand': 'Extra Virgin', 'type': 'Fats', 'expiry_date': '2025-07-15', 'info': None}


In [18]:
for item in Fruits_and_Vegetables:
    print(item.get_info())

{'name': 'Banana', 'brand': None, 'type': 'Fruits and Vegetables', 'expiry_date': '2024-12-23', 'info': None}
{'name': 'Apple', 'brand': None, 'type': 'Fruits and Vegetables', 'expiry_date': '2024-12-25', 'info': None}
{'name': 'Carrot', 'brand': None, 'type': 'Fruits and Vegetables', 'expiry_date': '2024-12-30', 'info': None}
{'name': 'Potato', 'brand': None, 'type': 'Fruits and Vegetables', 'expiry_date': '2025-01-10', 'info': None}


# Prompting for Ingredients base on your database

In [19]:
import time
class MealPlanner:
    def __init__(self, inventory: Inventory_Management):
        self.inventory = inventory
        self.selected_ingredients = {}  # Store selected Food_Management objects

    def pick_ingredients_by_category(self, category: str):
        
        sorted_items = self.inventory.sort_by_category_and_expiry()
        picked_ingredients = []

        if category not in sorted_items:
            print(f"Invalid category: {category}. Please choose a valid category.")
            return picked_ingredients

        items = sorted_items[category]
        print(f"\nCategory: {category}")
        if not items:
            print("No ingredients available in this category.")
            return picked_ingredients

        for i, item in enumerate(items, start=1):
            print(f"{i}. {item.get_info()}")
        time.sleep(1)

        while True:
            try:
                user_input = input(f"Enter the numbers of the ingredients you'd like to use from {category} (comma-separated): ").strip()
                if not user_input:
                    print("Input cannot be empty. Please enter valid indices.")
                    continue

                indices = [int(idx.strip()) for idx in user_input.split(",")]
                
                if any(idx < 1 or idx > len(items) for idx in indices):
                    print(f"Some indices are out of range. Please enter indices between 1 and {len(items)}.")
                    continue

                # Add selected Food_Management objects to the list
                picked_ingredients = [items[idx - 1] for idx in indices]
                break  # Exit loop if everything is valid

            except ValueError:
                print("Invalid input. Please enter numbers only, separated by commas.")


        # Add the selected ingredients to the selected_ingredients list
        self.selected_ingredients[category] = picked_ingredients
        
        print(f"\nSelected ingredients from the category '{category}':")
        picked_ingredients = [item.get_info() for item in picked_ingredients]
        print(picked_ingredients)
        
    
    def __str__(self):
        if not self.selected_ingredients:
            return "No ingredients selected yet."

        result = "Ingredients Provided:\n"
        for category in self.selected_ingredients:
            result += f"- {category}: "
            ingredients = self.selected_ingredients[category]
            result += "\n"
            if ingredients:
                result += ", ".join(
                    f"{ingredient._name} ({ingredient._brand if ingredient._brand else 'No Brand'}, expiry: {ingredient._expiry_date})"
                    for ingredient in ingredients
                )
            else:
                result += "No ingredients selected"
            result += "\n"
        
        return result


In [20]:
# Meal Planner
meal_planner = MealPlanner(inventory)

meal_planner.pick_ingredients_by_category("Carbohydrates")
meal_planner.pick_ingredients_by_category("Protein")
meal_planner.pick_ingredients_by_category("Fats")
meal_planner.pick_ingredients_by_category("Fruits and Vegetables")


Category: Carbohydrates
1. {'name': 'Bread', 'brand': 'Baker’s Delight', 'type': 'Carbohydrates', 'expiry_date': '2024-12-24', 'info': None}
2. {'name': 'Rice', 'brand': 'Golden Grain', 'type': 'Carbohydrates', 'expiry_date': '2026-05-01', 'info': None}

Selected ingredients from the category 'Carbohydrates':
[{'name': 'Rice', 'brand': 'Golden Grain', 'type': 'Carbohydrates', 'expiry_date': '2026-05-01', 'info': None}]

Category: Protein
1. {'name': 'Milk', 'brand': 'Dairy Pure', 'type': 'Protein', 'expiry_date': '2024-12-22', 'info': None}
2. {'name': 'Chicken Breast', 'brand': 'Farm Fresh', 'type': 'Protein', 'expiry_date': '2024-12-27', 'info': None}
3. {'name': 'Eggs', 'brand': 'Golden Eggs', 'type': 'Protein', 'expiry_date': '2024-12-29', 'info': None}
4. {'name': 'Yoghurt', 'brand': 'Marigold', 'type': 'Protein', 'expiry_date': '2025-01-13', 'info': None}
5. {'name': 'Cheese', 'brand': 'Cheddar Bliss', 'type': 'Protein', 'expiry_date': '2025-02-01', 'info': None}

Selected ingre

In [22]:
print(meal_planner)

Ingredients Provided:
- Carbohydrates: 
Rice (Golden Grain, expiry: 2026-05-01)
- Protein: 
Chicken Breast (Farm Fresh, expiry: 2024-12-27), Yoghurt (Marigold, expiry: 2025-01-13)
- Fats: 
Olive Oil (Extra Virgin, expiry: 2025-07-15)
- Fruits and Vegetables: 
Apple (No Brand, expiry: 2024-12-25), Carrot (No Brand, expiry: 2024-12-30)



# Chat Bot for Planning Meal today

In [23]:
system_prompt = f"""
You are a professional chef specializing in creating simple, nutritious meals. Use the following ingredients from the customer's fridge to assist them:

{str(meal_planner)}

Your tasks:
1. Suggest recipes using only these ingredients, be careful with the expiry dates of these ingredients.
2. Provide clear, step-by-step cooking instructions when asked.
3. Adapt recipes to user preferences or constraints during the chat.

Do not introduce ingredients outside this list. Keep track of the conversation history to continue assisting seamlessly.
If the customer do not have enough ingredients, suggest them to buy the missing ingredients.
"""


In [24]:
class MasterChef:
    def __init__(self, meal_planner: MealPlanner):
        self.meal_planner = meal_planner
        
        system_prompt = f"""
            You are a professional chef specializing in creating simple, nutritious meals. Use the following ingredients from the customer's fridge to assist them:

            {str(meal_planner)}

            Your tasks:
            1. Suggest recipes using only these ingredients, be careful with the expiry dates of these ingredients.
            2. Provide clear, step-by-step cooking instructions when asked.
            3. Adapt recipes to user preferences or constraints during the chat.
            4. Be user-friendly and engaging to keep the customer interested with your suggestions.

            Do not introduce ingredients outside this list. Keep track of the conversation history to continue assisting seamlessly.
            If the customer do not have enough ingredients, suggest them to buy the missing ingredients.
            """

        self.model = genai.GenerativeModel(
            "models/gemini-1.5-flash",
            system_instruction=system_prompt
        )
        self.chat = self.model.start_chat(history=[])

In [26]:
# Start the chatbot interaction
def chat_with_chef(master_chef):
    print("Welcome to MasterChef Chatbot! Let's discuss your meal plan.")
    print("Type 'exit' to end the conversation.\n")

    while True:
        # Get user input
        user_input = input("You: ").strip()
        
        # Exit condition
        if user_input.lower() == 'exit':
            print("Goodbye! Have a great day cooking!")
            break
        
        # Send user input to the chat model
        response = master_chef.chat.send_message(user_input)
        
        # Display chatbot response
        print(f"Chef: {response.text}")


In [27]:
master_chef = MasterChef(meal_planner)

# Start chatting
chat_with_chef(master_chef)

Welcome to MasterChef Chatbot! Let's discuss your meal plan.
Type 'exit' to end the conversation.

Chef: Given your fridge contents, I can suggest two simple and nutritious dinner options:


**Option 1:  Chicken and Rice with Sautéed Carrots**

This is a classic, comforting meal that's quick to prepare.  It uses the chicken breast, rice, carrots, and olive oil.

**Option 2:  Chicken and Apple Salad with Yogurt Dressing**

This is a lighter, refreshing option, perfect if you're looking for something less heavy.  It uses the chicken breast, apple, yoghurt, and a touch of olive oil for the dressing.


Which option sounds more appealing to you tonight?  Let me know and I can provide detailed instructions.

Chef: Okay, let's make **Chicken and Rice with Sautéed Carrots**! This recipe is easy and delicious.

**Yields:** 2 servings
**Prep time:** 10 minutes
**Cook time:** 25 minutes


**Ingredients:**

* 1 Chicken Breast (Farm Fresh)
* 1 cup Rice (Golden Grain)
* 2 medium Carrots (No Brand)
*

ValueError: Invalid input: 'content' argument must not be empty. Please provide a non-empty value.